In [ ]:
import json, pickle
from kLLMmeans import get_embeddings, summarize_cluster
from experiment_utils import load_dataset

file_path = "../kLLMmeans.py"  # Adjust path as needed
spec = importlib.util.spec_from_file_location("kLLMmeans", file_path)
kLLMmeans = importlib.util.module_from_spec(spec)
spec.loader.exec_module(kLLMmeans)

file_path = "../experiment_utils.py"  # Adjust path as needed
spec2 = importlib.util.spec_from_file_location("experiment_utils", file_path)
experiment_utils = importlib.util.module_from_spec(spec2)
spec2.loader.exec_module(experiment_utils)

In [ ]:
data_type = ['massive_D', 'massive_I', 'goemo', 'bank77', 'clinic']

for data in data_type:
    
    data_dict = {}
    print(data)
    
    labels, documents, num_clusters, prompt, text_type, instructor_prompt = experiment_utils.load_dataset(data, opt=data[-1])
    embeddings = {}
    for emb_type in ['distilbert', 'openai', 'e5-large', 'sbert']:
        print(emb_type)
        embeddings[emb_type] = kLLMmeans.get_embeddings(documents, emb_type=emb_type, instructor_prompt = "")
    data_dict = {}
    data_dict['data'] = data
    data_dict['labels'] = labels
    data_dict['num_clusters'] = num_clusters
    data_dict['documents'] = documents
    data_dict['embeddings'] = embeddings
    data_dict['prompt'] = prompt
    data_dict['text_type'] = text_type
        
    with open("../processed_data/data_" + data + ".pkl", "wb") as f:
        pickle.dump(data_dict, f)

    sentences_dict = {}
    
    for emb_type in ['distilbert', 'openai', 'e5-large', 'sbert']:
        print(emb_type)
        sents, index_map = build_sentence_corpus(documents)
        X, _ = embed_sentences(sents, emb_type = emb_type)
        text_sentences = {i: [] for i in range(len(documents))}
        for i in range(len(sents)):
            cur_index = index_map[i][0]
            text_sentences[cur_index].append([sents[i],X[i]])
        sentences_dict[emb_type] = text_sentences
        
        with open("../processed_data/data_sentences_" + data + ".pkl", "wb") as f:
            pickle.dump(sentences_dict, f)